<a href="https://colab.research.google.com/github/patriciogs/03MIAR-Algoritmos-de-Optimizacion-2024/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Patricio_Galdames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Patricio Galdames Sepúlveda  <br>
Url: https://github.com/patriciogs/03MIAR-Algoritmos-de-Optimizacion-2024/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Patricio_Galdames.ipynb<br>
Google Colab: https://colab.research.google.com/drive/1n80B0eM4BiHA5F1R9wlYiqV5QTxa9GZd?usp=sharing <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de una jornada de La Liga<br>
>3. Configuración de Tribunales

## Problema 1. Organizar sesiones de doblaje

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los datos son:



*   Número de actores: 10
*   Número de tomas: 30
*   Actores/Tomas: [matriz de 1s y 0s donde 1 indica que el actor participa en la toma y 0 en caso contrario]"

                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?


**¿Cómo represento el espacio de soluciones?**

El espacio de soluciones consiste en todas las posibles asignaciones de las 30 tomas a diferentes días, donde cada solución es una partición del conjunto total de tomas en subconjuntos (días). Cada toma debe estar asignada a exactamente un día, y la distribución debe respetar las restricciones del problema (no superar el número tomas diarias). La representación puede ser una lista donde cada elemento representa un día y contiene el conjunto de tomas asignadas a ese día.

**¿Cuál es la función objetivo?**

La función objetivo es minimizar la suma total de días de trabajo de todos los actores, donde:
- Un actor suma un día de trabajo cuando participa en al menos una toma de ese día
- El costo es independiente del número de tomas que el actor realice en un mismo día
- Se contabiliza la suma sobre todos los actores del total de días que cada uno debe asistir al estudio

**¿Cómo implemento las restricciones?**

Las restricciones del problema son:
1. No se pueden realizar más de 6 tomas por día
2. Para cada toma, deben estar presentes todos los actores que la matriz de participación indica con un 1
3. Cada toma debe ser asignada exactamente a un día



In [ ]:
#Respuesta


#Análisis
**¿Qué complejidad tiene el problema? Orden de complejidad y Contabilización del espacio de soluciones**

El problema requiere distribuir 30 tomas en diferentes días, donde cada solución debe satisfacer ciertas restricciones específicas. Para entender su complejidad, debemos examinar la naturaleza del espacio de soluciones y los factores que hacen que este problema sea computacionalmente desafiante.

**Espacio de Soluciones**

El problema corresponde a encontrar todas las posibles particiones de un conjunto de 30 tomas en subconjuntos (días), donde:
- Cada día puede contener entre 1 y 6 tomas
- Cada toma debe asignarse exactamente a un día
- El orden de las tomas dentro de un mismo día no es relevante
- El orden de los días no es relevante

El número mínimo de días corresponde a 5, que es el caso donde cada día se realizan 6 tomas, y el número máximo es de 30, que corresponde al escenario en el que cada día se realiza una toma. Este es un problema de partición de conjuntos con restricción en el tamaño de cada subconjunto.

La matriz de participación no reduce el espacio de soluciones válidas, ya que todas las particiones que cumplen con las restricciones de tamaño son posibles. La matriz solo afecta al costo de cada solución (número total de días que cada actor debe trabajar).

**Factores que Determinan la Complejidad**

La complejidad del problema está determinada por varios elementos interrelacionados:
1. La dimensión del problema: 30 tomas a distribuir
2. La restricción operativa: máximo 6 tomas por día
3. La naturaleza combinatoria: diferentes formas de particionar el conjunto de tomas
4. El cálculo del costo: para cada partición, se debe determinar el número total de días que cada actor debe trabajar



In [ ]:
#Respuesta


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Propongo utilizar una técnica heurística debido a la naturaleza combinatoria del problema y su alto espacio de soluciones. Específicamente, he diseñado tres posibles enfoques heurísticos:

1. **Bottom-up**: Este enfoque construye la solución incrementalmente desde cero, priorizando la optimización local en cada día. Para cada día, selecciona tomas considerando los actores que participan, maximizando el aprovechamiento diario (hasta 6 tomas) mientras busca minimizar los días totales de trabajo. La ventaja de este enfoque es que permite construir días eficientes desde el inicio.

2. **Top-down**: Este enfoque comienza con la solución menos eficiente (30 días, una toma por día) y mejora progresivamente mediante la fusión de días, respetando el límite de 6 tomas diarias. La ventaja es que cada fusión exitosa garantiza una reducción en el costo total.

3. **Híbrido**: Este enfoque combina las fortalezas de ambas estrategias, alternando entre la construcción de días optimizados (bottom-up) y la fusión de días existentes (top-down). Esto permite mayor flexibilidad en la búsqueda de soluciones óptimas.

La elección de una aproximación heurística se justifica por la complejidad del problema: encontrar la solución óptima requeriría explorar todas las posibles particiones de 30 tomas en grupos de 1 a 6 elementos, lo que resulta computacionalmente inviable para una solución exacta.


In [ ]:
# Importando librerias

import pandas as pd
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Lectura de archivo de planificacion
def leer_datos(archivo):
    """
    Lee el archivo Excel con la matriz de participación.
    Retorna una matriz numpy con los datos de participación, excluyendo la columna Total
    y la fila de totales.
    """
    df = pd.read_excel(archivo)

    # Excluimos la última columna (Total) y la última fila (totales)
    matriz_participacion = df.iloc[:-3, 1:-2].values

    # Verificamos las dimensiones correctas
    if matriz_participacion.shape != (30, 10):
        raise ValueError(f"La matriz debe ser de 30x10, pero es de {matriz_participacion.shape}")

    return matriz_participacion


In [ ]:
# Funciones auxiliares:
def calcular_coincidencias(toma, grupo_actual, matriz):
    """
    Calcula cuántos actores de la toma ya están en el grupo actual (la solución).
    Sirve para priorizar tomas que aprovechen actores ya presentes.
    """
    actores_toma = set([i for i in range(len(matriz[0])) if matriz[toma][i] == 1])
    actores_grupo = set()
    for t in grupo_actual:
        for i in range(len(matriz[0])):
            if matriz[t][i] == 1:
                actores_grupo.add(i)
    return len(actores_toma.intersection(actores_grupo))


def calcular_costo_total(solucion, matriz_participacion):
    """
    Calcula el costo total de la solución y muestra el desglose por actor.
    Retorna el costo total y un diccionario con los días de trabajo por actor.
    """
    n_actores = matriz_participacion.shape[1]
    dias_por_actor = [0] * n_actores

    # Para cada día en la solución (recuerdo: un dia tiene una o mas tomas)
    for dia in solucion:
        # Identificar qué actores trabajan en este día
        actores_del_dia = set()
        for toma in dia:
            for actor in range(n_actores):
                if matriz_participacion[toma][actor] == 1:
                    actores_del_dia.add(actor)

        # Incrementar el contador para cada actor que trabajó
        for actor in actores_del_dia:
            dias_por_actor[actor] += 1

    # Mostrar desglose por actor
    #print("\nDesglose de días de trabajo por actor:")
    #for actor in range(n_actores):
       # print(f"Actor {actor + 1}: {dias_por_actor[actor]} días")

    costo_total = sum(dias_por_actor)
    #print(f"\nCosto total (suma de todos los días de trabajo): {costo_total}")

    return costo_total, dias_por_actor

def visualizar_distribucion_trabajo(solucion, matriz_participacion, dias_por_actor):
    """
    Visualiza la distribución del trabajo de los actores utilizando la información
    previamente calculada de días por actor.

    Args:
        solucion: Lista donde cada elemento contiene las tomas asignadas a un día
        matriz_participacion: Matriz que indica la participación de actores en tomas
        dias_por_actor: Lista con el número de días que trabaja cada actor
    """
    n_actores = len(dias_por_actor)
    n_dias = len(solucion)

    print("\nDISTRIBUCIÓN DE TRABAJO POR DÍA Y ACTOR")
    print("=" * 80)

    # Creamos y rellenamos la matriz de trabajo para la visualización
    matriz_trabajo = [[False] * n_actores for _ in range(n_dias)]
    for dia_idx, dia in enumerate(solucion):
        for toma in dia:
            for actor in range(n_actores):
                if matriz_participacion[toma][actor] == 1:
                    matriz_trabajo[dia_idx][actor] = True

    # Mostramos el encabezado
    print(f"{'Día':^6}|", end="")
    for actor in range(n_actores):
        print(f" Actor {actor+1:2d} |", end="")
    print(f" Total Actores")
    print("-" * 80)

    # Mostramos la matriz de trabajo día por día
    for dia_idx in range(n_dias):
        print(f"{dia_idx+1:^6}|", end="")
        actores_del_dia = 0
        for actor in range(n_actores):
            trabaja = matriz_trabajo[dia_idx][actor]
            print(f"{'    X     ' if trabaja else '          '}|", end="")
            if trabaja:
                actores_del_dia += 1
        print(f"     {actores_del_dia:2d}")

    print("-" * 80)
    print(f"{'Total':^6}|", end="")
    for dias in dias_por_actor:
        print(f"     {dias:2d}   |", end="")
    print(f"     {sum(dias_por_actor):2d}")

    print("\nRESUMEN FINAL")
    print("=" * 80)
    print(f"Total de días de grabación: {n_dias}")
    print(f"Total de días-actor de trabajo: {sum(dias_por_actor)}")
    print("\nDistribución de carga de trabajo:")
    for actor, dias in enumerate(dias_por_actor):
        print(f"Actor {actor + 1}: {dias} días ({dias/n_dias*100:.1f}% de los días)")



In [ ]:
def heuristica_bottom_up(matriz_participacion):
    """
    Implementa la heurística bottom-up para agrupar las tomas en días.
    Retorna una lista de días, donde cada día es una lista de tomas.
    Las tomas se numeran de 0 a n-1 internamente.
    """
    # Verificar dimensiones de la matriz
    n_tomas = len(matriz_participacion)
    if n_tomas != 30:
        raise ValueError(f"Se esperaban 30 tomas, pero se encontraron {n_tomas}")
    n_tomas = len(matriz_participacion)
    tomas_sin_asignar = set(range(n_tomas))
    dias = []

    while tomas_sin_asignar:
        # Crear un nuevo día
        dia_actual = []

        while len(dia_actual) < 6 and tomas_sin_asignar:
            mejor_coincidencia = -1
            mejor_toma = None

            # Si el día está vacío, seleccionar la toma con más actores
            if not dia_actual:
                for toma in tomas_sin_asignar:
                    n_actores = sum(matriz_participacion[toma])
                    if mejor_toma is None or n_actores > mejor_coincidencia:
                        mejor_coincidencia = n_actores
                        mejor_toma = toma
            else:
                # Buscar la toma con más coincidencias con el grupo actual
                for toma in tomas_sin_asignar:
                    coincidencias = calcular_coincidencias(toma, dia_actual, matriz_participacion)
                    if coincidencias > mejor_coincidencia:
                        mejor_coincidencia = coincidencias
                        mejor_toma = toma

            # Si no encontramos ninguna toma compatible, tomamos cualquiera
            if mejor_toma is None:
                mejor_toma = tomas_sin_asignar.pop()
            else:
                tomas_sin_asignar.remove(mejor_toma)

            dia_actual.append(mejor_toma)

        dias.append(dia_actual)

    return dias


In [ ]:
# Código de prueba
# Leer datos del archivo Excel
file_directory = "/content/drive/MyDrive/Colab Notebooks"
os.chdir(file_directory)
print(os.listdir())

# Construir la ruta completa al archivo Excel
directorio_actual = os.path.abspath(os.getcwd())
archivo = "doblaje.xlsx"
print(f"Directorio actual (ruta absoluta): {directorio_actual}")


# Construir la ruta completa al archivo Excel
ruta_archivo = os.path.abspath(os.path.join(directorio_actual, archivo))
print(f"Ruta absoluta al archivo: {ruta_archivo}")

matriz = leer_datos(archivo)
print(matriz)
# Ejecutar la heurística
solucion = heuristica_bottom_up(matriz)

# Calcular y mostrar el costo total
costo_total, dias_por_actor = calcular_costo_total(solucion, matriz)

print("Solución encontrada:")
for i, dia in enumerate(solucion, 1):
    print(f"Día {i}: Tomas {[t+1 for t in dia]}")
print(f"Costo total (suma de días de trabajo): {costo_total}")

visualizar_distribucion_trabajo(solucion, matriz, dias_por_actor)


['Copia de Taller Evaluativo.ipynb', 'Copia de Algoritmos-Patricio_Galdames_Sepulveda-AG1 (1).ipynb', 'Algoritmos-Patricio_Galdames_Sepulveda-AG1.ipynb', 'Copia de Algoritmos-Patricio_Galdames_Sepulveda-AG1.ipynb', 'Copia de Algoritmos AG2(Divide y vencerás, Voraz, Backtracking) - Plantilla PAGS.ipynb', 'Algoritmos AG2(Divide y vencerás, Voraz, Backtracking) -Patricio Galdames Sepúlveda.ipynb', 'Algoritmos_AG3_Patricio_Galdames_Sepúlveda.ipynb', 'Algoritmos_AG3_Patricio_Galdames_Sepulveda.ipynb', ' Trabajo Práctico - Algoritmos(V2)-Patricio Galdames Sepúlveda.ipynb', 'Copia de AG3-Algoritmos(Colonia de Hormigas).ipynb', 'Algoritmos AG4 - Patricio_Galdames_Sepulveda.ipynb', 'Copia de  Trabajo Práctico - Algoritmos_Patricio_Galdames.ipynb', 'doblaje.xlsx', ' Trabajo Práctico - Algoritmos_Patricio_Galdames.ipynb']
Directorio actual (ruta absoluta): /content/drive/MyDrive/Colab Notebooks
Ruta absoluta al archivo: /content/drive/MyDrive/Colab Notebooks/doblaje.xlsx
[[ 1.  2.  3.  4.

## Solución Top-Down

In [ ]:
def heuristica_top_down(matriz_participacion):
    """
    Implementa la heurística top-down para agrupar las tomas en días.
    Comienza con una toma por día y luego intenta fusionar días.
    """
    # Inicializamos con la solución menos eficiente: una toma por día
    n_tomas = len(matriz_participacion)
    solucion = [[i] for i in range(n_tomas)]

    # Intentamos fusionar días mientras sea posible
    i = 0
    while i < len(solucion):
        j = i + 1
        while j < len(solucion):
            # Verificar si podemos fusionar los días i y j
            dia_fusionado = solucion[i] + solucion[j]

            # Si la fusión resulta en más de 6 tomas, no es válida
            if len(dia_fusionado) > 6:
                j += 1
                continue

            # Calculamos el costo antes y después de la fusión
            costo_antes = calcular_costo_total([solucion[i]], matriz_participacion)[0] + \
                         calcular_costo_total([solucion[j]], matriz_participacion)[0]

            # Creamos una solución temporal con la fusión para evaluar
            solucion_temp = solucion.copy()
            solucion_temp[i] = dia_fusionado
            solucion_temp.pop(j)

            costo_despues = calcular_costo_total([dia_fusionado], matriz_participacion)[0]

            # Si la fusión reduce el costo, la aplicamos
            if costo_despues <= costo_antes:
                solucion = solucion_temp
                # No incrementamos j porque ahora hay un nuevo día en esa posición
            else:
                j += 1
        i += 1

    return solucion

## Comentarios

El estudio y la implementación de tres diferentes enfoques heurísticos para resolver el problema de organización de sesiones de doblaje ha revelado aspectos intersantes. Los resultados de ejecutar las estrategias bottom-up, top-down e híbrida demuestra cómo diferentes aproximaciones al mismo problema pueden conducir a resultados significativamente distintos, incluso cuando aparentemente utilizan recursos similares.

El enfoque híbrido emergió como la solución más efectiva, alcanzando un costo total de 29 días de trabajo, una mejora notable frente a los 32 días logrados por la estrategia bottom-up y el resultado aún más costoso del enfoque top-down. Este éxito se puede atribuir a su capacidad única de combinar la construcción eficiente de grupos inicial del método bottom-up con la flexibilidad reorganizativa del top-down, complementada por una fase final de optimización que permite ajustes finos en la distribución de las tomas.

Esta experiencia subraya la importancia de no limitarse a un único enfoque cuando se abordan problemas complejos de optimización. La sinergia creada al combinar diferentes estrategias puede superar las limitaciones inherentes a cada método individual, llevando a soluciones más robustas y eficientes. En el contexto práctico de la planificación de sesiones de doblaje, esto se traduce en una organización más eficiente que minimiza los costos operativos mientras mantiene la viabilidad logística de las sesiones de grabación.



In [ ]:
def evaluar_solucion(solucion, matriz_participacion):
    """
    Evalúa una solución mostrando información detallada.
    """
    print("\nEvaluación de la solución encontrada:")
    print(f"Número total de días: {len(solucion)}")

    print("\nDistribución de tomas por día:")
    for i, dia in enumerate(solucion, 1):
        print(f"Día {i}: Tomas {[t+1 for t in dia]} ({len(dia)} tomas)")

    costo_total, dias_por_actor = calcular_costo_total(solucion, matriz_participacion)

    return costo_total, dias_por_actor

In [ ]:

    # Leemos los datos (ya estan leidos)
    #matriz = leer_datos("doblaje.xlsx")

    # Ejecutamos la heurística top-down
    solucion = heuristica_top_down(matriz)

    # Evaluamos y mostramos los resultados
    costo_total, dias_por_actor = evaluar_solucion(solucion, matriz)
    print(f"\nCosto total de la solución: {costo_total}")
    visualizar_distribucion_trabajo(solucion, matriz, dias_por_actor)


Evaluación de la solución encontrada:
Número total de días: 5

Distribución de tomas por día:
Día 1: Tomas [1, 2, 3, 4, 5, 6] (6 tomas)
Día 2: Tomas [7, 8, 9, 10, 11, 12] (6 tomas)
Día 3: Tomas [13, 14, 15, 16, 17, 18] (6 tomas)
Día 4: Tomas [19, 20, 21, 22, 23, 24] (6 tomas)
Día 5: Tomas [25, 26, 27, 28, 29, 30] (6 tomas)

Costo total de la solución: 38

DISTRIBUCIÓN DE TRABAJO POR DÍA Y ACTOR
 Día  | Actor  1 | Actor  2 | Actor  3 | Actor  4 | Actor  5 | Actor  6 | Actor  7 | Actor  8 | Actor  9 | Actor 10 | Total Actores
--------------------------------------------------------------------------------
  1   |    X     |    X     |    X     |    X     |    X     |          |    X     |    X     |          |          |      7
  2   |    X     |    X     |    X     |    X     |    X     |    X     |          |    X     |    X     |          |      8
  3   |    X     |    X     |    X     |    X     |    X     |    X     |    X     |          |          |    X     |      8
  4   |    X 

## Solución híbrida

In [ ]:
def heuristica_hibrida(matriz_participacion, porcentaje_bottom_up=0.6):
    """
    Implementa una heurística híbrida que combina bottom-up y top-down.
    Comienza construyendo días eficientes con bottom-up hasta alcanzar cierto porcentaje de tomas,
    luego usa top-down para las tomas restantes.

    Args:
        matriz_participacion: Matriz de participación de actores en tomas
        porcentaje_bottom_up: Porcentaje de tomas a manejar con bottom-up (default 60%)
    """
    n_tomas = len(matriz_participacion)
    tomas_bottom_up = int(n_tomas * porcentaje_bottom_up)

    # Fase 1: Bottom-up para las primeras tomas
    tomas_sin_asignar = set(range(n_tomas))
    dias = []

    # Construimos días usando bottom-up hasta alcanzar el porcentaje definido
    while len(tomas_sin_asignar) > (n_tomas - tomas_bottom_up):
        dia_actual = []

        while len(dia_actual) < 6 and tomas_sin_asignar and len(tomas_sin_asignar) > (n_tomas - tomas_bottom_up):
            mejor_coincidencia = -1
            mejor_toma = None

            # Selección de toma usando lógica bottom-up
            if not dia_actual:
                for toma in tomas_sin_asignar:
                    n_actores = sum(matriz_participacion[toma])
                    if mejor_toma is None or n_actores > mejor_coincidencia:
                        mejor_coincidencia = n_actores
                        mejor_toma = toma
            else:
                for toma in tomas_sin_asignar:
                    coincidencias = calcular_coincidencias(toma, dia_actual, matriz_participacion)
                    if coincidencias > mejor_coincidencia:
                        mejor_coincidencia = coincidencias
                        mejor_toma = toma

            if mejor_toma is None:
                mejor_toma = tomas_sin_asignar.pop()
            else:
                tomas_sin_asignar.remove(mejor_toma)

            dia_actual.append(mejor_toma)

        dias.append(dia_actual)

    # Fase 2: Top-down para las tomas restantes
    if tomas_sin_asignar:
        # Convertimos las tomas restantes en días individuales
        dias_restantes = [[toma] for toma in tomas_sin_asignar]

        # Aplicamos lógica top-down a los días restantes
        i = 0
        while i < len(dias_restantes):
            j = i + 1
            while j < len(dias_restantes):
                dia_fusionado = dias_restantes[i] + dias_restantes[j]

                if len(dia_fusionado) > 6:
                    j += 1
                    continue

                costo_antes = calcular_costo_total([dias_restantes[i]], matriz_participacion)[0] + \
                             calcular_costo_total([dias_restantes[j]], matriz_participacion)[0]

                solucion_temp = dias_restantes.copy()
                solucion_temp[i] = dia_fusionado
                solucion_temp.pop(j)

                costo_despues = calcular_costo_total([dia_fusionado], matriz_participacion)[0]

                if costo_despues <= costo_antes:
                    dias_restantes = solucion_temp
                else:
                    j += 1
            i += 1

        # Añadimos los días optimizados con top-down a nuestra solución
        dias.extend(dias_restantes)

    # Fase 3: Optimización final
    # Intentamos mejorar la solución completa mediante intercambios
    mejorado = True
    while mejorado:
        mejorado = False
        for i in range(len(dias)):
            for j in range(i + 1, len(dias)):
                # Intentamos intercambiar tomas entre días
                for toma_i in dias[i]:
                    for toma_j in dias[j]:
                        # Creamos días temporales con el intercambio
                        nuevo_dia_i = [t for t in dias[i] if t != toma_i] + [toma_j]
                        nuevo_dia_j = [t for t in dias[j] if t != toma_j] + [toma_i]

                        if len(nuevo_dia_i) <= 6 and len(nuevo_dia_j) <= 6:
                            costo_antes = calcular_costo_total([dias[i], dias[j]], matriz_participacion)[0]
                            costo_despues = calcular_costo_total([nuevo_dia_i, nuevo_dia_j], matriz_participacion)[0]

                            if costo_despues < costo_antes:
                                dias[i] = nuevo_dia_i
                                dias[j] = nuevo_dia_j
                                mejorado = True

    return dias

In [ ]:
# Leemos los datos, se asume leido
#matriz = leer_datos("doblaje.xlsx")

# Ejecutamos la heurística híbrida
solucion = heuristica_hibrida(matriz)

# Evaluamos y mostramos los resultados
print("\nSolución encontrada con heurística híbrida:")
for i, dia in enumerate(solucion, 1):
    print(f"Día {i}: Tomas {[t+1 for t in dia]}")

costo_total, dias_por_actor = calcular_costo_total(solucion, matriz)
print(f"\nCosto total de la solución híbrida: {costo_total}")

visualizar_distribucion_trabajo(solucion, matriz, dias_por_actor)


Solución encontrada con heurística híbrida:
Día 1: Tomas [7, 8, 12, 23, 26, 17]
Día 2: Tomas [5, 16, 4, 6, 10, 14]
Día 3: Tomas [11, 9, 21, 27, 13, 2]
Día 4: Tomas [18, 19, 20, 22, 1, 15]
Día 5: Tomas [24, 25, 28, 29, 30, 3]

Costo total de la solución híbrida: 29

DISTRIBUCIÓN DE TRABAJO POR DÍA Y ACTOR
 Día  | Actor  1 | Actor  2 | Actor  3 | Actor  4 | Actor  5 | Actor  6 | Actor  7 | Actor  8 | Actor  9 | Actor 10 | Total Actores
--------------------------------------------------------------------------------
  1   |    X     |    X     |    X     |    X     |    X     |          |          |    X     |          |    X     |      7
  2   |    X     |    X     |          |    X     |    X     |          |    X     |    X     |          |          |      6
  3   |    X     |    X     |    X     |    X     |    X     |    X     |          |          |    X     |          |      7
  4   |    X     |          |    X     |          |          |    X     |          |    X     |          

## Comentarios

El estudio y la implementación de tres diferentes enfoques heurísticos para resolver el problema de organización de sesiones de doblaje nos ha permitido encontrar soluciones eficientes para minimizar los costos de producción. Es importante destacar que todas nuestras soluciones utilizan 5 días físicos de grabación, que representa el mínimo posible dado el límite de 6 tomas por día para un total de 30 tomas.

La verdadera diferencia entre las soluciones radica en cómo distribuyen la participación de los actores en estos 5 días. Si consideramos que tenemos 10 actores y 5 días de grabación, el costo máximo teórico sería de 50 días-actor (si todos los actores tuvieran que asistir todos los días). En este contexto, el enfoque híbrido logró la distribución más eficiente, requiriendo solo 29 días-actor totales de asistencia. Esto significa que, en promedio, cada actor necesita asistir aproximadamente 3 días, aunque la distribución real varía según las necesidades específicas de cada personaje.

Esta optimización en la distribución de la participación de los actores representa un ahorro significativo comparado con soluciones menos eficientes, mientras mantiene la viabilidad logística de las sesiones de grabación. La clave del éxito del enfoque híbrido radica en su capacidad para combinar la construcción eficiente de grupos inicial con una reorganización flexible posterior, permitiendo encontrar patrones de asistencia que minimicen el número total de días que cada actor debe presentarse en el estudio.


# Análisis extra
En el enfoque híbrido de manera arbitraria fijamos que el 60% de las veces se emplee un enfoque bottom-up por sobre el de top-down. Creemos que el enfoque bottom-up es mejor al inicio del proceso porque permite construir días eficientes aprovechando las coincidencias entre actores cuando tenemos más tomas disponibles y, por lo tanto, más opciones para crear agrupaciones óptimas.

En cambio, top-down es mejor para manejar las tomas restantes, ya que su estrategia de fusionar días ayuda a encontrar combinaciones eficientes cuando quedan menos tomas y las opciones son más limitadas. Esta complementariedad entre ambos enfoques podría explicar porque su combinación encuentra un mejor óptimo, aprovechando las fortalezas de cada estrategia en diferentes etapas del proceso de optimización.

A continuación haremos un breve experimento para determinar si cambia la solución al modificar este parámetro y verificar si nuestra elección inicial del 60% es realmente la más efectiva para este problema específico.

In [ ]:
import time
from collections import defaultdict

def experimento_parametro(matriz_participacion, paso=0.1):
    """
    Realiza un experimento para encontrar el valor óptimo del parámetro porcentaje_bottom_up.

    Args:
        matriz_participacion: Matriz de participación de actores en tomas
        paso: Incremento entre valores del parámetro a probar (default 0.1 = 10%)
    """
    resultados = defaultdict(list)
    valores_parametro = [round(i * paso, 2) for i in range(1, int(1/paso))]

    print("Iniciando experimento...")
    print("=" * 50)

    for porcentaje in valores_parametro:
        print(f"\nProbando con porcentaje_bottom_up = {porcentaje:.2f}")

        # Realizamos múltiples pruebas para cada valor del parámetro
        for prueba in range(3):  # 3 pruebas por valor para tener un promedio
            inicio = time.time()
            solucion = heuristica_hibrida(matriz_participacion, porcentaje)
            tiempo = time.time() - inicio
            costo = calcular_costo_total(solucion, matriz_participacion)[0]

            resultados[porcentaje].append({
                'costo': costo,
                'tiempo': tiempo,
                'dias': len(solucion)
            })

            print(f"  Prueba {prueba + 1}: Costo = {costo}, Tiempo = {tiempo:.2f}s")

    # Analizar resultados
    print("\nResumen de resultados:")
    print("=" * 50)
    print(f"{'Porcentaje':^10} | {'Costo Promedio':^15} | {'Tiempo Promedio':^15} | {'Mejor Costo':^12}")
    print("-" * 60)

    mejor_porcentaje = None
    mejor_costo = float('inf')

    for porcentaje in valores_parametro:
        pruebas = resultados[porcentaje]
        costo_promedio = sum(p['costo'] for p in pruebas) / len(pruebas)
        tiempo_promedio = sum(p['tiempo'] for p in pruebas) / len(pruebas)
        mejor_costo_local = min(p['costo'] for p in pruebas)

        print(f"{porcentaje:^10.2f} | {costo_promedio:^15.2f} | {tiempo_promedio:^15.2f} | {mejor_costo_local:^12}")

        if mejor_costo_local < mejor_costo:
            mejor_costo = mejor_costo_local
            mejor_porcentaje = porcentaje

    print("\nConclusión:")
    print(f"El mejor valor encontrado para porcentaje_bottom_up es {mejor_porcentaje:.2f}")
    print(f"Con este valor se alcanzó un costo mínimo de {mejor_costo}")

    return resultados

In [ ]:
# Se asume que archivo participacion esta cargado (ha sido leido)
#matriz = leer_datos("doblaje.xlsx")
resultados = experimento_parametro(matriz, paso=0.1)

Iniciando experimento...

Probando con porcentaje_bottom_up = 0.10
  Prueba 1: Costo = 31, Tiempo = 0.09s
  Prueba 2: Costo = 31, Tiempo = 0.09s
  Prueba 3: Costo = 31, Tiempo = 0.09s

Probando con porcentaje_bottom_up = 0.20
  Prueba 1: Costo = 30, Tiempo = 0.09s
  Prueba 2: Costo = 30, Tiempo = 0.09s
  Prueba 3: Costo = 30, Tiempo = 0.09s

Probando con porcentaje_bottom_up = 0.30
  Prueba 1: Costo = 33, Tiempo = 0.06s
  Prueba 2: Costo = 33, Tiempo = 0.08s
  Prueba 3: Costo = 33, Tiempo = 0.06s

Probando con porcentaje_bottom_up = 0.40
  Prueba 1: Costo = 30, Tiempo = 0.07s
  Prueba 2: Costo = 30, Tiempo = 0.08s
  Prueba 3: Costo = 30, Tiempo = 0.06s

Probando con porcentaje_bottom_up = 0.50
  Prueba 1: Costo = 33, Tiempo = 0.06s
  Prueba 2: Costo = 33, Tiempo = 0.06s
  Prueba 3: Costo = 33, Tiempo = 0.06s

Probando con porcentaje_bottom_up = 0.60
  Prueba 1: Costo = 29, Tiempo = 0.10s
  Prueba 2: Costo = 29, Tiempo = 0.09s
  Prueba 3: Costo = 29, Tiempo = 0.09s

Probando con porcent